In [63]:
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.objects.petri.importer import importer as pnml_importer
from pm4py.objects.conversion.log import converter as log_converter
from pm4py.visualization.petrinet import visualizer as pn_visualizer
import datetime
import pandas as pd
import numpy as np


# Q4

### Import

In [2]:
no_appeals_log = xes_importer.apply("../../FMP_no_appeals_cleaned.xes")

no_appeals_net, no_appeals_initial_marking, no_appeals_final_marking = pnml_importer.apply("../results/im_no_appeals.pnml")


In [3]:
appeals_log = xes_importer.apply("../../FMP_appeals.xes")

appeals_net, appeals_initial_marking, appeals_final_marking = pnml_importer.apply("../results/im_appeals.pnml")


In [10]:
credit_collection_log = xes_importer.apply("../../FMP_appeals_credit_collection.xes")

credit_df = log_converter.apply(credit_collection_log, variant=log_converter.Variants.TO_DATA_FRAME)


In [125]:
appeals_duration_log = xes_importer.apply("../../FMP_appeals_duration.xes")

appeals_df = log_converter.apply(appeals_duration_log, variant=log_converter.Variants.TO_DATA_FRAME)


---
a)

In [33]:
# only keep send for credit collection events
filtered_df = credit_df[credit_df["concept:name"] == "Send for Credit Collection"]

filtered_df.head()


,concept:instance,amount,time:timestamp,article,points,duration,org:resource,dismissal,concept:name,vehicleClass,case_duration,totalPaymentAmount,lifecycle:transition,case:concept:name,expense,notificationType,lastSent,matricola,paymentAmount
8,412,NaN,2009-03-29 22:00:00+00:00,NaN,NaN,498.958333,NaN,NaN,Send for Credit Collection,NaN,654.0,NaN,complete,A13617,NaN,NaN,NaN,NaN,NaN
17,475,NaN,2010-10-14 22:00:00+00:00,NaN,NaN,819.000000,NaN,NaN,Send for Credit Collection,NaN,1137.0,NaN,complete,A14652,NaN,NaN,NaN,NaN,NaN
26,502,NaN,2009-03-29 22:00:00+00:00,NaN,NaN,507.958333,NaN,NaN,Send for Credit Collection,NaN,663.0,NaN,complete,A14910,NaN,NaN,NaN,NaN,NaN
32,529,NaN,2009-03-29 22:00:00+00:00,NaN,NaN,760.958333,NaN,NaN,Send for Credit Collection,NaN,960.0,NaN,complete,A1516,NaN,NaN,NaN,NaN,NaN
38,556,NaN,2010-10-14 22:00:00+00:00,NaN,NaN,953.958333,NaN,NaN,Send for Credit Collection,NaN,1140.0,NaN,complete,A15307,NaN,NaN,NaN,NaN,NaN


In [34]:
#compute stats for the event duration
filtered_df[["duration"]].describe()


,duration
count,412.000000
mean,690.893811
std,268.384542
min,39.000000
25%,502.958333
50%,643.000000
75%,838.000000
max,1594.958333


In [109]:
#check what influence it has when there are two credit collections per year

def perform_credit_collection_at(dates, minimum_wait):
    new_durations = []
    for index, row in filtered_df.iterrows():
        timestamp = row["time:timestamp"]
        duration = row["duration"]
        minimum = duration
        
        timestamp = timestamp + pd.DateOffset(days=-duration)
    
        for date in dates:
            delta = (date - timestamp.astimezone(None)).days

            if delta < minimum_wait:
                continue
        
            if abs(delta) < minimum:
                minimum = abs(delta)

        new_durations.append(minimum)
    
    print(np.mean(new_durations))
    

In [120]:
#check impact of yearly cycle
yearly_dates = [pd.Timestamp(2000, 1, 1, 0),
pd.Timestamp(2001, 1, 1, 0),
pd.Timestamp(2002, 1, 1, 0),
pd.Timestamp(2003, 1, 1, 0),
pd.Timestamp(2004, 1, 1, 0),
pd.Timestamp(2005, 1, 1, 0),
pd.Timestamp(2006, 1, 1, 0),
pd.Timestamp(2007, 1, 1, 0),
pd.Timestamp(2008, 1, 1, 0),
pd.Timestamp(2009, 1, 1, 0),
pd.Timestamp(2010, 1, 1, 0),
pd.Timestamp(2011, 1, 1, 0),
pd.Timestamp(2012, 1, 1, 0),
pd.Timestamp(2013, 1, 1, 0),
pd.Timestamp(2014, 1, 1, 0)]

perform_credit_collection_at(yearly_dates, 0)

186.47825647249192


In [114]:
#check impact of half-yearly cycle
half_yearly_dates = [pd.Timestamp(2000, 1, 1, 0),
pd.Timestamp(2000, 6, 1, 0),
pd.Timestamp(2001, 1, 1, 0),
pd.Timestamp(2001, 6, 1, 0),
pd.Timestamp(2002, 1, 1, 0),
pd.Timestamp(2002, 6, 1, 0),
pd.Timestamp(2003, 1, 1, 0),
pd.Timestamp(2003, 6, 1, 0),
pd.Timestamp(2004, 1, 1, 0),
pd.Timestamp(2004, 6, 1, 0),
pd.Timestamp(2005, 1, 1, 0),
pd.Timestamp(2005, 6, 1, 0),
pd.Timestamp(2006, 1, 1, 0),
pd.Timestamp(2006, 6, 1, 0),
pd.Timestamp(2007, 1, 1, 0),
pd.Timestamp(2007, 6, 1, 0),
pd.Timestamp(2008, 1, 1, 0),
pd.Timestamp(2008, 6, 1, 0),
pd.Timestamp(2009, 1, 1, 0),
pd.Timestamp(2009, 6, 1, 0),
pd.Timestamp(2010, 1, 1, 0),
pd.Timestamp(2010, 6, 1, 0),
pd.Timestamp(2011, 1, 1, 0),
pd.Timestamp(2011, 6, 1, 0),
pd.Timestamp(2012, 1, 1, 0),
pd.Timestamp(2012, 6, 1, 0),
pd.Timestamp(2013, 1, 1, 0),
pd.Timestamp(2013, 6, 1, 0),
pd.Timestamp(2014, 1, 1, 0),
pd.Timestamp(2014, 6, 1, 0)]

perform_credit_collection_at(half_yearly_dates, 0)

95.1626213592233


In [122]:
#in a last check we perform a half-yearly cycle that has a minimum waiting time of 1 year

perform_credit_collection_at(yearly_dates, 180)

perform_credit_collection_at(half_yearly_dates, 180)


351.6533171521036
266.6019417475728


In [123]:
#in a last check we perform a half-yearly cycle that has a minimum waiting time of 1/2 year

perform_credit_collection_at(yearly_dates, 365)

perform_credit_collection_at(half_yearly_dates, 365)


502.1695995145631
440.87115695792886


In [130]:
#calculate mean waiting time for the send appeal event
send_appeal_df = appeals_df[appeals_df["concept:name"] == "Send Appeal to Prefecture"]

send_appeal_df[["event_duration"]].describe()


,event_duration
count,4141.000000
mean,203.530045
std,559.544717
min,0.000000
25%,9.000000
50%,24.000000
75%,44.000000
max,4221.041667


In [132]:
#calculate mean waiting time for the send appeal event
notify_appeal_df = appeals_df[appeals_df["concept:name"] == "Notify Result Appeal to Offender"]

notify_appeal_df[["event_duration"]].describe()

,event_duration
count,863.000000
mean,32.393299
std,36.826151
min,0.000000
25%,7.000000
50%,14.000000
75%,47.979167
max,193.958333


---
b)

In [6]:
parameters = {pn_visualizer.Variants.PERFORMANCE.value.Parameters.FORMAT: "png"}

no_appeals_performance_viz = pn_visualizer.apply(no_appeals_net, no_appeals_initial_marking, no_appeals_final_marking, parameters=parameters, variant=pn_visualizer.Variants.PERFORMANCE, log=no_appeals_log)
pn_visualizer.save(no_appeals_performance_viz, "no_appeals_performance.png")


In [7]:
appeals_performance_viz = pn_visualizer.apply(appeals_net, appeals_initial_marking, appeals_final_marking, parameters=parameters, variant=pn_visualizer.Variants.PERFORMANCE, log=appeals_log)
pn_visualizer.save(appeals_performance_viz, "appeals_performance.png")
